在这一小节你将学到：
1. 多层感知机MLP是什么
2. MLP的隐藏层hidden layer是什么，什么是激活函数
- 隐藏层位于输入层和输出层之间，不直接与外界交互，是使用激活函数的地方
- 激活函数是引入非线性能力的函数，有了激活函数 → 网络可以近似任何函数（万能逼近定理）

3. 如何训练一个多层感知机(简单实例)
- 前向传播Forward是什么
- 损失函数/代价函数 LossFunction (Cost Function)
- backward反向传播算法
- 随机梯度下降SGD
- 学习率

4. 如何调用Pytorch快速实现神经网络并完成训练

# 2.多层感知机MLP
## 2.1 激活函数
如第一小节小结所提，多层感知机能处理非线性的能力，来自于隐藏层的激活函数。我们先通过一些例子来理解最简单，最常用的两个激活函数：Sigmoid，ReLU；从而理解为什么线性的输入层，经过了激活函数，就有了表示非线性的能力。


## 2.2 MLP怎么构建呢？

## 2.3 MLP的训练过程⭐

## 前向传播

| 步骤    | 输入            | 核心动作        | 产出             |
| ----- | ------------- | ----------- | -------------- |
| 1. 前向 | 数据 + 参数       | 矩阵运算 + 激活函数 | 预测结果           |
| 2. 损失 | 预测结果 + 真值     | 计算误差函数      | Loss值          |
| 3. 反向 | Loss值 + 链式法则  | 求导（找责任）     | 梯度 (Gradient)  |
| 4. 更新 | 参数 + 梯度 + 学习率 | 梯度下降（减法）    | 新参数 (New w, b) |


## 附录内容
### 其他优化算法（近似最小损失函数）

- **Mini-batch SGD**  
目前最常用的方法。每次处理一小部分数据（如 1024 个样本，称为一个 Batch），计算平均梯度后更新参数。这种方式兼顾了速度和矩阵运算的效率。

- **Adam 优化器**  
目前 90% 以上的大模型（如 GPT 系列、Llama、DeepSeek）训练时都采用 Adam（Adaptive Moment Estimation）。它为梯度下降增加了两个“buff”：

    - **动量（Momentum）**：不仅考虑当前的梯度，还参考之前的惯性。直观理解：如果之前一直往南走，现在的坡度也向南，就加速冲下去；如果坡度突然变化，也不会立刻掉头，而是受惯性影响平滑转弯。这有助于跨越一些微小的“局部最小值”。
    - **自适应学习率（Adaptive Learning Rate）**：为每个参数（如几千亿个 $w$）配备独立的“变速箱”。直观理解：更新频繁、梯度大的参数，学习率调小，避免步子太大；更新慢、梯度小的参数，学习率调大，提高效率。

这些优化算法让模型训练更加高效、稳定。